<a href="https://colab.research.google.com/github/SaidMuratbekov/hello-world/blob/main/Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import sys
#rootpath = '/content/drive/MyDrive/ColabNotebooks/mipt_2_sem/graph_ml/data/'
#sys.path.append(rootpath)
import lightgbm as lgb
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import requests
from zlib import adler32

assert nx.__version__ == '2.5.1'

url = 'https://raw.githubusercontent.com/netspractice/ml-on-graphs/main/competition_link_prediction/features.txt'
open('features.txt', 'wb').write(requests.get(url).content)
features_path = url

url = 'https://raw.githubusercontent.com/netspractice/ml-on-graphs/main/competition_link_prediction/labeled_edges.txt'
open('labeled_edges.txt', 'wb').write(requests.get(url).content);
labeled_edges_path = url

url = 'https://raw.githubusercontent.com/netspractice/ml-on-graphs/main/competition_link_prediction/unlabeled_edges.txt'
open('unlabeled_edges.txt', 'wb').write(requests.get(url).content);
unlabeled_edges_path = url

#with open('features.txt') as f:
#    print(f.read()[:190])
#with open('labeled_edges.txt') as f:
#    print(f.read()[:190])
#with open('unlabeled_edges.txt') as f:
#    print(f.read()[:190])



# features = pd.read_csv(features_path, skiprows=0, index_col=0, delimiter="\t")
features = pd.read_csv(features_path, delimiter=" ", index_col=0, header=None)
print(features.shape)
features.head()

labeled_edges = pd.read_csv(labeled_edges_path, delimiter=" ", header=None)
print(labeled_edges.shape)
labeled_edges.head()

unlabeled_edges = pd.read_csv(unlabeled_edges_path, delimiter=" ", header=None)
print(unlabeled_edges.shape)
unlabeled_edges.head()

features_nodes_left = features.loc[list(labeled_edges[0])]
features_nodes_right = features.loc[list(labeled_edges[1])]

# features_nodes_right

def dfs_row_by_row_dot_product(df1, df2):
    return np.array(np.matmul(np.matrix(df1), np.matrix(df2).T).diagonal()).flatten()
    # list(np.array(np.matmul(np.matrix(df1), np.matrix(df2).T).diagonal()).flatten())
    # np.matmul(np.array(df1), np.array(df2).T).diagonal()

labeled_edges['similarity'] = dfs_row_by_row_dot_product(features_nodes_left, features_nodes_right)
labeled_edges

features_nodes_left_test = features.loc[list(unlabeled_edges[0])]
features_nodes_right_test = features.loc[list(unlabeled_edges[1])]
unlabeled_edges['similarity'] = dfs_row_by_row_dot_product(features_nodes_left_test, features_nodes_right_test)
unlabeled_edges

#Make new train dataframe: nodeleft features + noder features + target (0/1 - connected or not)

x_test = pd.DataFrame(unlabeled_edges['similarity'])
x_train = pd.DataFrame(labeled_edges['similarity'])
y_train = np.array(list(labeled_edges[2]))


lgb_train = lgb.Dataset(x_train, label=y_train.reshape(-1))
lgb_val = lgb_train

params = {
    "metric":"mape",
    "num_threads": -1,
    "objective": "regression",
    "verbosity": 1,
    "rounds":50,
    "verbose_eval":50,
    "early_stopping_rounds":1000,
    "lambda_l2":30,
    "lambda_l1":4,
    "min_gain_to_split":1,
    "num_leaves":31
}
rounds = 10


evals_result = {}
model = lgb.train(
            params = params,
            #lgb_train,
            train_set = lgb_train,
            num_boost_round = rounds,
            valid_sets = [lgb_train,lgb_val],
            verbose_eval=True,
            early_stopping_rounds=False,
            callbacks=[lgb.record_evaluation(evals_result)] #same -> evals_result=evals_result,
            #callbacks=[lgb.print_evaluation(period=1, show_stdv=True)] #same -> evals_result=evals_result,
        )
steps = model.best_iteration #print(f"Best: {steps}") #print('lgb fit done')
history_callback = evals_result

y_test = model.predict(x_test)
y_test

y_test_bool = np.where(y_test > 0.9, 1, 0)

y_test_bool[0:10]
len(y_test_bool)

submit = pd.DataFrame(y_test_bool)

submit.to_csv('D:\MSAI\Semester 2\Graph\GRAPH_CONTEST-5.txt', sep=' ', encoding='utf-8', index=False, header=None)

(3312, 3703)
(6048, 3)
(3024, 2)
